In [1]:
from pathlib import Path

In [5]:
text = Path("input.txt").read_text()
len(text)

1115393

In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [12]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


Typically **tiktoken** from openAI and **sentencepiece** from google used to encode words. These are subword encodings.

In [26]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115393]) torch.int64


In [18]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [27]:
torch.manual_seed(1234)
batch_size = 4 # how many individual sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[1:1+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("----")
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        #print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[47, 56, 57, 58,  1, 15, 47, 58],
        [47, 56, 57, 58,  1, 15, 47, 58],
        [47, 56, 57, 58,  1, 15, 47, 58],
        [47, 56, 57, 58,  1, 15, 47, 58]])
targets:
torch.Size([4, 8])
tensor([[42,  1, 51, 39, 56, 56, 47, 39],
        [ 1, 51, 43, 52,  1, 41, 39, 52],
        [52,  1, 63, 43, 58,  6,  1, 44],
        [ 5, 57, 63,  1, 61, 53, 56, 58]])
----


In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1234)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, target):
        logits = self.token_embedding_table(idx) # B, T, C
        
        return logits